# Working with missing data

In this section, we will discuss missing (also referred to as `NA`) values in cudf. cudf supports having missing values in all dtypes. These missing values are represented by `<NA>`. These values are also referenced as "null values".

## How to Detect missing values

To detect missing values, you can use `isna()` and `notna()` functions.

In [1]:
import cudf
import numpy as np

In [2]:
df = cudf.DataFrame({'a': [1, 2, None, 4], 'b':[0.1, None, 2.3, 17.17]})

In [3]:
df

,a,b
0,1,0.1
1,2,<NA>
2,<NA>,2.3
3,4,17.17


In [4]:
df.isna()

,a,b
0,False,False
1,False,True
2,True,False
3,False,False


In [5]:
df['a'].notna()

0     True
1     True
2    False
3     True
Name: a, dtype: bool

One has to be mindful that in Python (and NumPy), the nan's don’t compare equal, but None's do. Note that pandas/NumPy uses the fact that `np.nan != np.nan`, and treats `None` like `np.nan`.



In [6]:
None == None

True

In [7]:
np.nan == np.nan

False

So as compared to above, a scalar equality comparison versus a None/np.nan doesn’t provide useful information.



In [8]:
df['b'] == np.nan

0    False
1    False
2    False
3    False
Name: b, dtype: bool

## Float dtypes and missing data

Because ``NaN`` is a float, a column of integers with even one missing values is cast to floating-point dtype. However this doesn't happen by default.

By default if a ``NaN`` value is passed to `Series` constructor, it is treated as `<NA>` value. 

In [9]:
cudf.Series([1, 2, np.nan])

0       1
1       2
2    <NA>
dtype: int64

Hence to consider a ``NaN`` as ``NaN`` you will have to pass `nan_as_null=False` parameter into `Series` constructor.

In [10]:
cudf.Series([1, 2, np.nan], nan_as_null=False)

0    1.0
1    2.0
2    NaN
dtype: float64

## Datetimes

For `datetime64` types, cudf doesn't support having `NaT` values. Instead these values which are specific to numpy and pandas are considered as null values(`<NA>`) in cudf. The actual underlying value of `NaT` is `min(int64)` and cudf retains the underlying value when converting a cudf object to pandas object.



In [11]:
import pandas as pd
datetime_series = cudf.Series([pd.Timestamp("20120101"), pd.NaT, pd.Timestamp("20120101")])
datetime_series

0    2012-01-01 00:00:00.000000
1                          <NA>
2    2012-01-01 00:00:00.000000
dtype: datetime64[us]

In [12]:
datetime_series.to_pandas()

0   2012-01-01
1          NaT
2   2012-01-01
dtype: datetime64[ns]

any operations on rows having `<NA>` values in `datetime` column will result in `<NA>` value at the same location in resulting column:

In [13]:
datetime_series - datetime_series

0    0 days 00:00:00
1               <NA>
2    0 days 00:00:00
dtype: timedelta64[us]

## Inserting missing data

All dtypes support insertion of missing value by assignment. Any specific location in series can made null by assigning it to `None`.

In [14]:
series = cudf.Series([1, 2, 3, 4])

In [15]:
series

0    1
1    2
2    3
3    4
dtype: int64

In [16]:
series[2] = None

In [17]:
series

0       1
1       2
2    <NA>
3       4
dtype: int64

## Calculations with missing data

Null values propagate naturally through arithmetic operations between pandas objects.

In [18]:
df1 = cudf.DataFrame({'a':[1, None, 2, 3, None], 'b':cudf.Series([np.nan, 2, 3.2, 0.1, 1], nan_as_null=False)})

In [19]:
df2 = cudf.DataFrame({'a':[1, 11, 2, 34, 10], 'b':cudf.Series([0.23, 22, 3.2, None, 1])})

In [20]:
df1

,a,b
0,1,NaN
1,<NA>,2.0
2,2,3.2
3,3,0.1
4,<NA>,1.0


In [21]:
df2

,a,b
0,1,0.23
1,11,22.0
2,2,3.2
3,34,<NA>
4,10,1.0


In [22]:
df1 + df2

,a,b
0,2,NaN
1,<NA>,24.0
2,4,6.4
3,37,<NA>
4,<NA>,2.0


While summing the data, `NA` values will be treated as `0`.

In [23]:
df1['a'].sum()

6

Since `NA` values are treated as `0`, the mean would result to 2 in this case `(1 + 0 + 2 + 3 + 0)/5 = 2`

In [24]:
df1['a'].mean()

2.0

Cumulative methods like `cumsum` and `cumprod` ignore `NA` values by default.

In [25]:
df1['a'].cumsum()

0       1
1    <NA>
2       3
3       6
4    <NA>
Name: a, dtype: int64

To preserve `NA` values in cumulative methods, provide `skipna=False`.

In [26]:
df1['a'].cumsum(skipna=False)

0       1
1    <NA>
2    <NA>
3    <NA>
4    <NA>
Name: a, dtype: int64

## Sum/product of Null/nans

The sum of an empty or all-NA Series of a DataFrame is 0.

In [27]:
cudf.Series([np.nan], nan_as_null=False).sum()

0.0

In [28]:
cudf.Series([], dtype='float64').sum()

0.0

The product of an empty or all-NA Series of a DataFrame is 1.

In [29]:
cudf.Series([np.nan], nan_as_null=False).prod()

1.0

In [30]:
cudf.Series([], dtype='float64').prod()

1.0

## NA values in GroupBy

`NA` groups in GroupBy are automatically excluded. For example:

In [31]:
df1

,a,b
0,1,NaN
1,<NA>,2.0
2,2,3.2
3,3,0.1
4,<NA>,1.0


In [32]:
df1.groupby('a').mean()

,b
a,
2,3.2
1,NaN
3,0.1


## Filling missing values: fillna

`fillna()` can fill in `NA` values with non-NA data.

In [33]:
df1

,a,b
0,1,NaN
1,<NA>,2.0
2,2,3.2
3,3,0.1
4,<NA>,1.0


In [34]:
df1['b'].fillna(10)

0    10.0
1     2.0
2     3.2
3     0.1
4     1.0
Name: b, dtype: float64

## Filling with cudf Object

You can also fillna using a dict or Series that is alignable. The labels of the dict or index of the Series must match the columns of the frame you wish to fill. The use case of this is to fill a DataFrame with the mean of that column.

In [35]:
import cupy as cp
dff = cudf.DataFrame(cp.random.randn(10, 3), columns=list('ABC'))

In [36]:
dff.iloc[3:5, 0] = np.nan

In [37]:
dff.iloc[4:6, 1] = np.nan

In [38]:
dff.iloc[5:8, 2] = np.nan

In [39]:
dff

,A,B,C
0,0.893770,-0.187920,-0.723388
1,0.847884,-0.207499,-0.382658
2,-0.387564,-1.734471,0.905929
3,NaN,-0.659590,-0.050078
4,NaN,NaN,0.760201
5,1.652248,NaN,NaN
6,-0.107916,-1.280595,NaN
7,-0.428468,0.709294,NaN
8,-0.030085,-0.392301,0.175510
9,0.606390,-0.530527,0.162528


In [40]:
dff.fillna(dff.mean())

,A,B,C
0,0.893770,-0.187920,-0.723388
1,0.847884,-0.207499,-0.382658
2,-0.387564,-1.734471,0.905929
3,0.380782,-0.659590,-0.050078
4,0.380782,-0.535451,0.760201
5,1.652248,-0.535451,0.121149
6,-0.107916,-1.280595,0.121149
7,-0.428468,0.709294,0.121149
8,-0.030085,-0.392301,0.175510
9,0.606390,-0.530527,0.162528


In [41]:
dff.fillna(dff.mean()[1:3])

,A,B,C
0,0.893770,-0.187920,-0.723388
1,0.847884,-0.207499,-0.382658
2,-0.387564,-1.734471,0.905929
3,NaN,-0.659590,-0.050078
4,NaN,-0.535451,0.760201
5,1.652248,-0.535451,0.121149
6,-0.107916,-1.280595,0.121149
7,-0.428468,0.709294,0.121149
8,-0.030085,-0.392301,0.175510
9,0.606390,-0.530527,0.162528


## Dropping axis labels with missing data: dropna

Missing data can be excluded using `dropna()`:



In [42]:
df1

,a,b
0,1,NaN
1,<NA>,2.0
2,2,3.2
3,3,0.1
4,<NA>,1.0


In [43]:
df1.dropna(axis=0)

,a,b
2,2,3.2
3,3,0.1


In [44]:
df1.dropna(axis=1)

""
0
1
2
3
4


An equivalent `dropna()` is available for Series. 

In [45]:
df1['a'].dropna()

0    1
2    2
3    3
Name: a, dtype: int64

## Replacing generic values

Often times we want to replace arbitrary values with other values.

`replace()` in Series and `replace()` in DataFrame provides an efficient yet flexible way to perform such replacements.

In [46]:
series = cudf.Series([0.0, 1.0, 2.0, 3.0, 4.0])

In [47]:
series

0    0.0
1    1.0
2    2.0
3    3.0
4    4.0
dtype: float64

In [48]:
series.replace(0, 5)

0    5.0
1    1.0
2    2.0
3    3.0
4    4.0
dtype: float64

You can replace a list of values by a list of other values:

In [49]:
series.replace([0, 1, 2, 3, 4], [4, 3, 2, 1, 0])

0    4.0
1    3.0
2    2.0
3    1.0
4    0.0
dtype: float64

You can also specify a mapping dict:

In [50]:
series.replace({0: 10, 1: 100})

0     10.0
1    100.0
2      2.0
3      3.0
4      4.0
dtype: float64

For a DataFrame, you can specify individual values by column:

In [51]:
df = cudf.DataFrame({"a": [0, 1, 2, 3, 4], "b": [5, 6, 7, 8, 9]})

In [52]:
df

,a,b
0,0,5
1,1,6
2,2,7
3,3,8
4,4,9


In [53]:
df.replace({"a": 0, "b": 5}, 100)

,a,b
0,100,100
1,1,6
2,2,7
3,3,8
4,4,9


## String/regular expression replacement

cudf supports replacing string values using `replace` API:

In [54]:
d = {"a": list(range(4)), "b": list("ab.."), "c": ["a", "b", np.nan, "d"]}

In [55]:
df = cudf.DataFrame(d)

In [56]:
df

,a,b,c
0,0,a,a
1,1,b,b
2,2,.,<NA>
3,3,.,d


In [57]:
df.replace(".", "A Dot")

,a,b,c
0,0,a,a
1,1,b,b
2,2,A Dot,<NA>
3,3,A Dot,d


Replace a few different values (list -> list):


In [58]:
df.replace(["a", "."], ["b", "--"])

,a,b,c
0,0,b,b
1,1,b,b
2,2,--,<NA>
3,3,--,d


Only search in column 'b' (dict -> dict):

In [59]:
df.replace({"b": "."}, {"b": "replacement value"})

,a,b,c
0,0,a,a
1,1,b,b
2,2,replacement value,<NA>
3,3,replacement value,d


## Numeric replacement

`replace()` can also be used similar to `fillna()`.

In [60]:
df = cudf.DataFrame(cp.random.randn(10, 2))

In [61]:
df[np.random.rand(df.shape[0]) > 0.5] = 1.5

In [62]:
df.replace(1.5, None)

,0,1
0,<NA>,<NA>
1,-0.19561263,-1.17183202
2,0.228898835,-0.175869569
3,-1.350722518,1.41702518
4,-1.523589596,0.53646915
5,-1.133955666,-0.155280322
6,<NA>,<NA>
7,<NA>,<NA>
8,<NA>,<NA>
9,<NA>,<NA>


Replacing more than one value is possible by passing a list.



In [63]:
df00 = df.iloc[0, 0]

In [64]:
df.replace([1.5, df00], [5, 10])

,0,1
0,5.000000,5.000000
1,-0.195613,-1.171832
2,0.228899,-0.175870
3,-1.350723,1.417025
4,-1.523590,0.536469
5,-1.133956,-0.155280
6,5.000000,5.000000
7,5.000000,5.000000
8,5.000000,5.000000
9,5.000000,5.000000


You can also operate on the DataFrame in place:



In [65]:
df.replace(1.5, None, inplace=True)

In [66]:
df

,0,1
0,<NA>,<NA>
1,-0.19561263,-1.17183202
2,0.228898835,-0.175869569
3,-1.350722518,1.41702518
4,-1.523589596,0.53646915
5,-1.133955666,-0.155280322
6,<NA>,<NA>
7,<NA>,<NA>
8,<NA>,<NA>
9,<NA>,<NA>
